# word2vec 개선 1

- CBOW모델은 단어 2개를 맥락으로 사용해 이를 바탕으로 하나의 단어를 추측함
- 입력 측 가중치와의 행렬 곱으로 은닉층 계산
- 출력 측 가중치 행렬 곱으로 점수 구함
- 점수에 소프트맥스 함수 적용해 출현 확률 도출
- 이 확률을 정답 레이블과 비교 (교차 엔트로피 오차 적용) 손실 구함

병목을 초래하는 요소
- 입력층의 원핫 표현과 가중치 행렬 W_in 곱 계산
    -> embedding 계층 도입으로 해결
- 은닉층과 가중치 행렬 W_out의 곱과 softmax 계층 계산
    -> negative sampling 도입으로 해결 

## Embedding 계층

- 실질적으로는 행렬의 특정 행을 추출하는 연산만 수행됨
- 따라서 원핫 변환과 matmul 계층의 행렬 곱 계산을 필요 없음

In [2]:
import numpy as np
W = np.arange(21).reshape(7,3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [4]:
# W로 부터 원하는 여러 행은 다음과 같이 추출 가능함
idx = np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [5]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

- 순전파: 가중치 W의 특정 행 추출
- 역전파: 앞 층으로 부터 전해진 기울기를 그대로 흘려 보냄

In [6]:
def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    dW[self.idx] = dout
    return None
# 해당 방식을 사용하면 idx 원소가 중복될 때 중복 할당되며 덮어씌워짐

In [7]:
# 할당이 아닌 더하기를 통해 해결
def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    
    for i, word_id in enumerate(self.idx):
        dW[word_id] +=dout[i]
    return None

# W와 같은 크기의 dW 행렬을 만들어 둘 필요는 없지만 이렇게 한 이유는 갱신용 클래스 Optimizer와 조합해 사용하기 위해서임

# word2vec 개선 2

## 은닉층 이후 계산의 문제점
- 은닉층의 뉴런과 가중치 행렬 (W_out)의 곱
- softmax 계층 계산

## 다중 분류에서 이진 분류로

- Yes/No로 답변 가능한 질문으로 변환 필요
    -> 맥락이 you와 goodbye 일 때, 타깃 단어는 say입니까? 라는 질문에 답하는 신경망을 고안해야 함
- 이전의 출력층에서는 모든 단어를 대상으로 계산을 수행했다면, 여기서는 say 라는 단어 하나에 주목해 그 점수만을 계산하는 차이 있음

## 시그모이드 함수와 교차 엔트로피 오차

- 이진 분류: 점수에 시그모이드 함수를 적용해 확률로 변환
- 손실함수: 교차 엔트로피 오차

## 다중 분류에서 이진 분류로(구현)

In [ ]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis = 1)
        
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

## 네거티브 샘플링

- 긍정적인 예에 대해서는 sigmoid 계층의 출력을 1에 가깝게 만들고, 부정적 예에 대해서는 sigmoid 계층의 출력을 0에 가깝게 만들어야 함
- 모든 부정적인 예를 대상으로 이진분류를 학습: 어휘 수가 늘어나면 감닫ㅇ 불가
- 부정적인 예를 몇개 선택하는 네거티브 샘플링 도입
- 네거티브 샘플링 기법은 긍정, 부정 예의 손실을 더한 값을 최종 손실로 함

## 네거티브 샘플링의 샘플링 기법
- 말뭉치의 통계 데이터를 기초로 샘플링
- 말뭉치에서 자주 등장하는 단어를 많이 추출하고, 드물게 등장하는 단어를 적게 추출함
- 희소한 단어는 선택되기 어려움


In [1]:
import numpy as np

# 0에서 9까지의 숫자 중 하나를 무작위 샘플링
np.random.choice(10) 

4

In [2]:
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'hello'

In [3]:
# 5개 샘플링 (중복 있음)
np.random.choice(words, size=5)

array(['I', '.', '.', 'say', 'hello'], dtype='<U7')

In [4]:
# 5개 샘플링 (중복 없음)
np.random.choice(words, size=5, replace = False)

array(['say', '.', 'I', 'goodbye', 'hello'], dtype='<U7')

In [5]:
# 확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

- 확률에 0.75 제곱을 하고, 변형된 확률 분포의 합으로 나누어 사용
- 이는 출현 확률이 낮은 단어를 버리지 않기 위함임
- 0.75 제곱을 통해 원래 확률 낮은 단어의 확률을 살짝 올릴 수 있음
- 0.75에 이론적 의미는 없으므로 다른 값 사용해도 됨

In [6]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


Unigramsampler class의 인수
- corpus: id 목록
- power: 제곱할 값
- sample_size: 네거티브 샘플링 수행 횟수

Unigramsampler class의 메서드
- get_negative_sample: target 인수로 지정한 단어를 긍정적 예로 해석하고, 그 외 단어 ID를 샘플링 함

In [1]:
from negative_sampling_layer import UnigramSampler

In [2]:

corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1,3,0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[2 3]
 [1 2]
 [2 1]]


## 네거티브 샘플링 구현

In [4]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power = 0.75, sample_size = 5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] # +1 for positive
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
        self.params , self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)
        
        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]  # embed_dot에 해당하는 타겟이라는 의미인 듯
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        
        return dh       

# 개선판 word2vec 학습

## CBOW 모델 구현

In [2]:
import sys
sys.path.append('..')
from common.np import *
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

In [3]:
class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # 레이어 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        
        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs1 = W_in
        self.word_vecs2 = W_out
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)  # average
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

## CBOW 모델 학습 코드

In [10]:
! pip uninstall dataset

^C


In [16]:
import os
os.getcwd()

'C:\\Users\\ykm25\\DLFromScratch2\\Chap04-Word2Vec_Improved'

In [23]:
! pip uninstall dataset

^C


In [27]:
import sys
sys.path.append('..')
sys.path.append('C:\\Users\\ykm25\\DLFromScratch2\\')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from datasets import ptb

In [32]:
# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 1

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
# if config.GPU:
#     contexts, target = to_gpu(contexts), to_gpu(target)

In [33]:
# 모델 등 생성
model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

In [ ]:
# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size, eval_interval=2000)
trainer.plot()

| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 41.59


## 모델 평가

In [ ]:
import sys
sys.path.append('..')
import pickle
from common.util import most_similar, analogy

In [ ]:
pkl_file = './cbow_params.pkl'
with open(pkl_file, 'rb') as f:
    params = pickle.load(f)

In [ ]:
word_vecs = params['word_vecs']
word_to_id = params['word_to_id']
id_to_word = params['id_to_word']

In [ ]:
# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

In [ ]:
# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)